In [1]:
import numpy as np
import pandas as pd
import seaborn, time

from pomegranate import BayesianNetwork

ModuleNotFoundError: No module named 'pomegranate'

In [2]:
seaborn.set_style('whitegrid')

In [3]:
"""
X = np.random.randint(2, size=(2000, 7))
X[:,3] = X[:,1]
X[:,6] = X[:,1]

X[:,0] = X[:,2]

X[:,4] = X[:,5]

model = BayesianNetwork.from_samples(X, algorithm='exact')
print(model.structure)
model.plot()
"""

"\nX = np.random.randint(2, size=(2000, 7))\nX[:,3] = X[:,1]\nX[:,6] = X[:,1]\n\nX[:,0] = X[:,2]\n\nX[:,4] = X[:,5]\n\nmodel = BayesianNetwork.from_samples(X, algorithm='exact')\nprint(model.structure)\nmodel.plot()\n"

In [4]:
def sanitize_celeb_attr_data(unsanitize_attrs_file):
    
    attr_list = []
    
    unsanitize_attrs = open(unsanitize_attrs_file,'r').readlines()
    
    for a in unsanitize_attrs:
        attr_line = a.strip('\n').split(' ')
        attrs_sanitized = [attr for attr in attr_line if attr != '']
        attr_list.append(attrs_sanitized)
        
    with open('/home/c01muya/vae-disentanglement/datasets/celeba/list_attr_celeba_sanitized.txt','w') as list_attr_celeb_sen:
        for attr_line in attr_list:
            list_attr_celeb_sen.write(
                ",".join(attr_line) + "\n"
            )

In [5]:
#sanitize_celeb_attr_data("/home/c01muya/vae-disentanglement/datasets/celeba/list_attr_celeba.txt")

In [40]:
celeb_attrs = pd.read_csv("/home/c01muya/vae-disentanglement/datasets/celeba/list_attr_celeba_sanitized.txt", 
                          sep=",", encoding='ascii')
celeb_attrs.drop('Image_ID', axis='columns', inplace=True)
celeb_attrs.head()

,5_o_Clock_Shadow,Arched_Eyebrows,Attractive,Bags_Under_Eyes,Bald,Bangs,Big_Lips,Big_Nose,Black_Hair,Blond_Hair,...,Sideburns,Smiling,Straight_Hair,Wavy_Hair,Wearing_Earrings,Wearing_Hat,Wearing_Lipstick,Wearing_Necklace,Wearing_Necktie,Young
0,-1,1,1,-1,-1,-1,-1,-1,-1,-1,...,-1,1,1,-1,1,-1,1,-1,-1,1
1,-1,-1,-1,1,-1,-1,-1,1,-1,-1,...,-1,1,-1,-1,-1,-1,-1,-1,-1,1
2,-1,-1,-1,-1,-1,-1,1,-1,-1,-1,...,-1,-1,-1,1,-1,-1,-1,-1,-1,1
3,-1,-1,1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,1,-1,1,-1,1,1,-1,1
4,-1,1,1,-1,-1,-1,1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,1,-1,-1,1


In [41]:

hair_color_conditions = [
    celeb_attrs['Black_Hair'] == 1,
    celeb_attrs['Brown_Hair'] == 1,
    celeb_attrs['Blond_Hair'] == 1,
    celeb_attrs['Gray_Hair']  == 1,
    (celeb_attrs['Black_Hair'] != 1) & (celeb_attrs['Brown_Hair'] != 1) & (celeb_attrs['Blond_Hair'] != 1) & (celeb_attrs['Gray_Hair']  != 1)
]

hair_color_values = ['Black', 'Brown', 'Blond', 'Gray', 'Unknown']
celeb_attrs['Hair_Color'] = np.select(hair_color_conditions, hair_color_values)
#celeb_attrs[ ['Hair_Color','Black_Hair', 'Brown_Hair', 'Blond_Hair', 'Gray_Hair'] ]
celeb_attrs.drop(['5_o_Clock_Shadow', 'Black_Hair','Brown_Hair','Blond_Hair','Gray_Hair'], axis='columns', inplace=True)

In [42]:
hair_type_conditions = [
    celeb_attrs['Bald'] == 1,
    celeb_attrs['Bangs'] == 1,
    celeb_attrs['Straight_Hair'] == 1,
    celeb_attrs['Wavy_Hair']  == 1,
    (celeb_attrs['Bald'] != 1) & (celeb_attrs['Bangs'] != 1) & (celeb_attrs['Straight_Hair'] != 1) & (celeb_attrs['Wavy_Hair'] != 1)
]

hair_type_values = ['Bald', 'Bangs', 'Straight', 'Wavy','Unknown']

celeb_attrs['Hair_Type'] = np.select(hair_type_conditions, hair_type_values)
#celeb_attrs[ ['Hair_Type','Bald', 'Bangs', 'Straight_Hair', 'Wavy_Hair'] ]
celeb_attrs.drop(['Bald', 'Bangs', 'Straight_Hair', 'Wavy_Hair'], axis='columns', inplace=True)

In [44]:
celeb_attrs.head(10)

,Arched_Eyebrows,Attractive,Bags_Under_Eyes,Big_Lips,Big_Nose,Blurry,Bushy_Eyebrows,Chubby,Double_Chin,Eyeglasses,...,Sideburns,Smiling,Wearing_Earrings,Wearing_Hat,Wearing_Lipstick,Wearing_Necklace,Wearing_Necktie,Young,Hair_Color,Hair_Type
0,1,1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,1,1,-1,1,-1,-1,1,Brown,Straight
1,-1,-1,1,-1,1,-1,-1,-1,-1,-1,...,-1,1,-1,-1,-1,-1,-1,1,Brown,Unknown
2,-1,-1,-1,1,-1,1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,1,Unknown,Wavy
3,-1,1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,1,-1,1,1,-1,1,Unknown,Straight
4,1,1,-1,1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,1,-1,-1,1,Unknown,Unknown
5,1,1,-1,1,-1,-1,-1,-1,-1,-1,...,-1,-1,1,-1,1,-1,-1,1,Brown,Wavy
6,-1,1,1,1,1,-1,1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,1,Black,Straight
7,1,-1,1,1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,1,Black,Unknown
8,1,1,-1,1,-1,-1,-1,-1,-1,-1,...,-1,1,1,-1,1,-1,-1,1,Unknown,Bangs
9,-1,1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,1,-1,-1,1,Unknown,Wavy


In [45]:
celeb_attrs_mat = celeb_attrs.to_numpy()
print(celeb_attrs_mat.shape)

(202599, 33)


In [46]:
celeb_attrs_mat_subset = celeb_attrs_mat[:10000,:]
print(celeb_attrs_mat_subset.shape)

(10000, 33)


In [47]:
model = BayesianNetwork.from_samples(celeb_attrs_mat_subset, 
                                     state_names=celeb_attrs.columns.to_list(), 
                                     algorithm='chow-liu')
print(model.structure)

((), (27,), (4,), (0,), (13,), (1,), (31,), (4,), (7,), (1,), (17,), (27,), (27,), (27,), (24,), (17,), (14,), (13,), (12,), (12,), (11,), (31,), (11,), (17,), (12,), (13,), (31,), (0,), (13,), (13,), (1,), (13,), (27,))


In [49]:
#%matplotlib inline

model.plot("../reports/learned-dag-3.pdf")

In [ ]:
# 1. The labels in celeba don't capture the variation within each attribute value. Not all blond hair at the same for example. 
# So we shouldn't use just a categorical to represent that information.


In [2]:
adj_mat = [(), (27,), (4,), (0,), (13,), (1,), (31,), (4,), (7,), (1,), 
           (17,), (27,), (27,), (27,), (24,), (17,), (14,), (13,), (12,), 
           (12,), (11,), (31,), (11,), (17,), (12,), (13,), (31,), (0,), 
           (13,), (13,), (1,), (13,), (27,)]



In [31]:
parents = [3, 27]
children = [1, 11, 12, 13, 32]
get_layer_mask(parents, children, adj_mat)

parents = [1, 11, 12, 13, 32]
children = [4, 5, 9, 17, 18, 19, 20, 22, 24, 25, 28, 29, 30, 31]
get_layer_mask(parents, children, adj_mat)



array([[0., 0., 0., 0., 0.],
       [1., 1., 1., 1., 1.]])

In [2]:
cd D:\Saarbrucken\EDA_Research\thesis_code\disentanglement_lib_pl

D:\Saarbrucken\EDA_Research\thesis_code\disentanglement_lib_pl


In [4]:
import torch
import numpy as np
import pandas as pd
import seaborn, time
import torch.nn.functional as F

from common import dag_utils
from common.special_modules import DAGInteractionLayer

In [5]:
adj_mat = [(), (27,), (4,), (0,), (13,), (1,), (31,), (4,), (7,), (1,), 
           (17,), (27,), (27,), (27,), (24,), (17,), (14,), (13,), (12,), 
           (12,), (11,), (31,), (11,), (17,), (12,), (13,), (31,), (0,), 
           (13,), (13,), (1,), (13,), (27,)]

In [6]:
parents = [3, 27]
children = [1, 11, 12, 13, 32]

dag_layer = DAGInteractionLayer(
    parents_list=parents, 
    children_list=children, 
    adjacency_matrix=adj_mat, 
    interm_unit_dim=2, 
    bias=True
)

In [7]:
dag_layer.diagnostic_forward(
    torch.randn(size=(4,2))
)

Input:  tensor([[-0.7045, -0.4278],
        [ 0.6620,  0.4411],
        [ 0.2410,  1.7375],
        [-0.9837, -0.7455]])
mask_input_to_interm:  tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]])
masked_input_to_interm: tensor([[ 0.0000,  0.0000, -0.0000,  0.0000, -0.0000, -0.0000,  0.0000, -0.0000,
         -0.0000, -0.0000],
        [-1.7341,  0.4121,  0.8381, -0.7316, -0.5100, -1.3836, -0.5140, -0.0806,
          1.0158, -0.3585]], grad_fn=<MulBackward0>)
masked_input_to_interm.Size(): torch.Size([2, 10])
X * W_1 tensor([[ 0.7419, -0.1763, -0.3585,  0.3130,  0.2182,  0.5919,  0.2199,  0.0345,
         -0.4346,  0.1534],
        [-0.7648,  0.1818,  0.3697, -0.3227, -0.2249, -0.6102, -0.2267, -0.0356,
          0.4480, -0.1581],
        [-3.0130,  0.7161,  1.4562, -1.2711, -0.8860, -2.4039, -0.8931, -0.1401,
          1.7650, -0.6229],
        [ 1.2928, -0.3072, -0.6248,  0.5454,  0.3802,  1.0314,  0.3832,  0.0601,
         -0.7573,  0.

tensor([[0.0000, 0.0623, 0.0000, 0.1017, 0.0065],
        [0.0000, 0.0986, 0.0000, 0.0000, 0.1301],
        [0.0000, 0.3885, 0.0000, 0.0000, 0.5125],
        [0.0000, 0.1085, 0.0000, 0.1772, 0.0114]], grad_fn=<ReluBackward0>)

In [1]:
cd D:\Saarbrucken\EDA_Research\thesis_code\disentanglement_lib_pl

D:\Saarbrucken\EDA_Research\thesis_code\disentanglement_lib_pl


In [2]:
from collections import namedtuple
from models import cs_vae

NetworkArgs = namedtuple("NetworkArgs", ["encoder","decoder","interm_unit_dim", 
                                         "z_dim", "num_channels", "image_size"])
network_args = NetworkArgs(
    ['SimpleGaussianConv64'],
    ['SimpleConv64'],
    2,
    10,
    3,
    64
)
adjacency_matrix = [(), (27,), (4,), (0,), (13,), (1,), (31,), (4,), (7,), (1,), 
           (17,), (27,), (27,), (27,), (24,), (17,), (14,), (13,), (12,), 
           (12,), (11,), (31,), (11,), (17,), (12,), (13,), (31,), (0,), 
           (13,), (13,), (1,), (13,), (27,)]
test_vae = cs_vae.ConceptStructuredVAE(adjacency_matrix, network_args)

print(test_vae)

[[0], [3, 27], [1, 11, 12, 13, 32], [4, 5, 9, 17, 18, 19, 20, 22, 24, 25, 28, 29, 30, 31], [2, 6, 7, 10, 14, 15, 21, 23, 26], [8, 16]]
ConceptStructuredVAE(
  (encoder): SimpleGaussianConv64(
    (main): Sequential(
      (0): Conv2d(3, 32, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
      (1): ReLU(inplace=True)
      (2): Conv2d(32, 32, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
      (3): ReLU(inplace=True)
      (4): Conv2d(32, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
      (5): ReLU(inplace=True)
      (6): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
      (7): ReLU(inplace=True)
      (8): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
      (9): ReLU(inplace=True)
      (10): Conv2d(256, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
      (11): ReLU(inplace=True)
      (12): Flatten3D()
      (13): Linear(in_features=256, out_features=20, bias=True)
    )
  )
  (dag_network): Sequential(
    (0): 

In [3]:
test_vae

ConceptStructuredVAE(
  (encoder): SimpleGaussianConv64(
    (main): Sequential(
      (0): Conv2d(3, 32, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
      (1): ReLU(inplace=True)
      (2): Conv2d(32, 32, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
      (3): ReLU(inplace=True)
      (4): Conv2d(32, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
      (5): ReLU(inplace=True)
      (6): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
      (7): ReLU(inplace=True)
      (8): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
      (9): ReLU(inplace=True)
      (10): Conv2d(256, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
      (11): ReLU(inplace=True)
      (12): Flatten3D()
      (13): Linear(in_features=256, out_features=20, bias=True)
    )
  )
  (dag_network): Sequential(
    (0): DAGInteractionLayer (1 -> 2 -> 2, parents: [0], children: [3, 27], )
    (1): DAGInteractionLayer (2 -> 2 -> 5, parents: [3, 27], child

In [4]:
test_vae.dag_layer_nodes

[[0],
 [3, 27],
 [1, 11, 12, 13, 32],
 [4, 5, 9, 17, 18, 19, 20, 22, 24, 25, 28, 29, 30, 31],
 [2, 6, 7, 10, 14, 15, 21, 23, 26],
 [8, 16]]